# KD線交易策略驗證
Author：余慶龍  
Establish Date：2021.7.25  
Last Modified Date：2021.9.10

## Table of Contents
1. 讀取檔案模塊
2. 指標與交易策略
3. 回測模塊
4. 執行回測並搜集資料
5. 統計結果

## 1. 讀取檔案模塊

In [2]:
"""
把csv或excel檔案轉換成dataframe
以供後續的pandas操作
"""

import pandas as pd
import numpy as np
from datetime import datetime

def readStock_file(file, filetype='csv'):


    if filetype == 'excel':
        df = pd.read_excel(file, engine='openpyxl', parse_dates=True, header=None)
    else:
        df = pd.read_csv(file)

    # 取代原本的 column 名稱
    # 檔案的日期與開高低收需要照這個順序
    colume_name = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.columns = colume_name

    # 用日期這一行當做 df 的索引
    df = df.set_index('Date')

    # 把日期轉成 datetime的格式(從string)
    df.index = pd.to_datetime(df.index)

    # 照日期排序並把空資料轉成numpy的nan
    df = df.sort_index()
    df = df.replace(r'^\s*-$', np.nan, regex=True)


    for col in df.columns:
        if(col=='Date'):
            continue;
        df[col] = np.array([float(x) for x in df[col]])

    return df

def random_sample(ticker_list_file,  column = '證券代碼', times = 10):
    

    ticker_list_df = pd.read_csv(ticker_list_file, encoding='utf8', usecols=[column])
    ticker_list = ticker_list_df[column].to_list()
    
    sample = np.random.choice(ticker_list, times)
    sample_list = [i.replace(' ', '') for i in sample.tolist()]

    return sample_list

def file_list_with_directory(file_list, path):
    
    files = [path+i+'.csv' for i in file_list]
    return files

def get_dataframe(data_file, days=0):
    ticker = pd.read_csv(data_file)

    # change the name of columns
    ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # set the column "Date" to index
    ticker = ticker.set_index('Date')

    # set index column("Date") from string to datetime.datetime
    ticker.index = pd.to_datetime(ticker.index)

    # sort index column("Date") chronologically
    ticker = ticker.sort_index()
    
    if(days):
        ticker = ticker.tail(days)
    return ticker

## 2. 指標與交易策略

In [28]:
# Define constants
KD_n = 9
upperBound = 80
lowerBound = 20
middleBound = 50

def kd_indicator(ticker):

    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    data = ticker.copy(deep=True)

    tmp['max_close'] = ticker['Close'].rolling(KD_n, min_periods=1).max()
    tmp['min_close'] = ticker['Close'].rolling(KD_n, min_periods=1).min()

    # RSV also called FastK
    tmp['RSV'] = (ticker['Close'] - tmp['min_close'])/(tmp['max_close']-tmp['min_close'])*100

    data['k'] = np.zeros(ticker.shape[0])
    data['d'] = np.zeros(ticker.shape[0])

    # skip first day
    # because RSV of first day is NaN
    for i in range(1, tmp.shape[0]):

        # means data['k'][i]
        data.loc[(data.index[i], 'k')] = data['k'][i-1] * (2/3) + tmp['RSV'][i] * (1/3)
        data.loc[(data.index[i], 'd')] = data['d'][i-1] * (2/3) + data['k'][i] * (1/3)
    
    return data

In [45]:
def plot_trading(data, bound='normal'):
    
    plots = []
    
    # the position in graph
    buy, sell = [], []

    # Adjust the position to avoid signals block the lines
    for index, row in data.iterrows():
        buy.append(row['Close']*0.985 if row['buy'] else np.nan)
        sell.append(row['Close']*1.01 if row['sell'] else np.nan)


    plots = [ 
        # Add the KD indicator to plot
        #     '#c8d164' is yellow
        #     '#7dc2fa' is blue
        mpf.make_addplot(data['k'], panel = 1, color='#c8d164', title='K(yellow), D(blue)'),
        mpf.make_addplot(data['d'], panel = 1, color='#7dc2fa'),
    ]
    
    # Add upperbound & lowerbound to plot
    if bound == 'normal':
        plots += [
            mpf.make_addplot(np.full((len(data['k'])), upperBound), panel = 1, color='#bf5ce0', alpha=0.5),
            mpf.make_addplot(np.full((len(data['d'])), lowerBound), panel = 1, color='#7e3696', alpha=0.5),
        ]
    elif bound == 'middle':
        plots += [
            mpf.make_addplot(np.full((len(data['k'])), middleBound), panel = 1, color='#bf5ce0', alpha=0.5),
        ]  

    # Add trading point to plot
    #    np.isnan(sell) determine the value is nan or not
    #    count_nonzero(np.isnan(sell)) counts how many nan we have
    #    if nan amount equal the length of array, we don't have sell record
    if not np.count_nonzero(np.isnan(sell)) == len(sell):
        plots.append(mpf.make_addplot(sell , type = 'scatter', color = 'red', marker = 'v', markersize = 100))
    if not np.count_nonzero(np.isnan(buy)) == len(buy):
        plots.append(mpf.make_addplot(buy , type = 'scatter', color = '#cfc01d', marker = '^', markersize = 200))

    mpf.plot( data, 
                type='candle',       # candlestick chart of stock price
                volume=True,         # plots trading volume as well
                volume_panel=2,
                figscale=2,          # make graph twice bigger
                style='yahoo',       # color scheme using 'yahoo' style
                addplot = plots,
    )    

In [55]:
import mplfinance as mpf

def bound_cross(ticker):
    
    # skip first KD_n days
    for i in range(KD_n, data.shape[0]):

        if(data['k'][i] < 20):

            # k upcross d
            if((data['k'][i-1] < data['d'][i-1]) and
               (data['k'][i]   > data['d'][i]) and
               (data['k'][i-1] < data['k'][i])):

                data.loc[(data.index[i], 'buy')] = 1

        if(data['k'][i] > 80):

            # k downcross d
            if((data['k'][i-1] > data['d'][i-1]) and
               (data['k'][i]   < data['d'][i]) and
               (data['k'][i-1] > data['k'][i])):

                data.loc[(data.index[i], 'sell')] = 1

    return data

def middle_line(ticker):
    
    cool_down_days = 0
    # skip first KD_n days
    for i in range(KD_n, data.shape[0]):

        if(data['k'][i] > data['d'][i] > 50 and
           cool_down_days <= 0):

            data.loc[(data.index[i], 'buy')] = 1

            # reset cooldown
            cool_down_days = KD_n


        if(data['k'][i] < data['d'][i] < 50 and
           cool_down_days <= 0):

            data.loc[(data.index[i], 'sell')] = 1

            # reset cooldown
            cool_down_days = KD_n

        cool_down_days-=1

    return data

# V means volume
def bound_cross_V(ticker):
    
    data['vol_5'] = data['Volume'].rolling(5).mean()


    # skip first KD_n days
    for i in range(KD_n, data.shape[0]):

        if(data['Volume'][i] > data['vol_5'][i]):

            # k upcross d
            if((data['k'][i-1] < data['d'][i-1]) and
               (data['k'][i]   > data['d'][i]) and
               (data['k'][i-1] < data['k'][i])):

                data.loc[(data.index[i], 'buy')] = 1

        if(data['k'][i] > 80):

            # k downcross d
            if((data['k'][i-1] > data['d'][i-1]) and
               (data['k'][i]   < data['d'][i]) and
               (data['k'][i-1] > data['k'][i])):

                data.loc[(data.index[i], 'sell')] = 1

    return data


# V means volume
# M means moving stop loss
def bound_cross_VM(ticker):
   
    data['vol_5'] = data['Volume'].rolling(5).mean()


    top = 0
    moving_stop_loss_ratio = 0.1
    holding_shares = 0
    # skip first KD_n days
    for i in range(KD_n, data.shape[0]):

        if(data['Volume'][i] > data['vol_5'][i]):

            # k upcross d
            if((data['k'][i-1] < data['d'][i-1]) and
               (data['k'][i]   > data['d'][i]) and
               (data['k'][i-1] < data['k'][i])):

                data.loc[(data.index[i], 'buy')] = 1
                holding_shares += 1
              

        if(data['Close'][i] < top*(1-moving_stop_loss_ratio)):


            data.loc[(data.index[i], 'sell')] = 1
            holding_shares = 0
            top = 0


        if(data['Close'][i] > top and holding_shares):
            top = data['Close'][i]
            
    return data

## 3. 回測模塊

In [51]:
from termcolor import colored
def backtesting(data):
    # how many shares we hold, and the money balance
    have_shares , balance = 0, 0

    # multiple buy and 1 sell as "a trade"
    # record trading times and how many trade are making money
    make_money, trade_times = 0, 0
    
    if np.count_nonzero(data['buy']) == 0:
        return np.nan, np.nan
    
    # we use geometic average
    # this is products of return rate
    return_rate = 1
    for index, row in data.iterrows():

        if(row['buy'] == 1):
            balance -= row['Close']
            have_shares += 1
            print(colored('\tbuy ', 'green'),'|', colored(index.strftime("%Y-%m-%d"), 'green'), '|', colored(row['Close'], 'green'))

        # sell signal or it reach ends
        elif((data.loc[index]['sell'] == 1 or index == data.index.values[-1])and
        # and we have some shares
              have_shares != 0):
            print(colored('\tsell', 'red'),'|', colored(index.strftime("%Y-%m-%d"), 'red'), '|', colored(row['Close'],'red'), "*" , have_shares)
            

            principal = -balance
            balance += row['Close'] * have_shares

            # to calculate accuracy
            trade_times += 1
            if(balance > 0):
                make_money += 1

            # plus 1 avoiding get too small
            return_rate *= 1+(balance / principal)

            # reset balance & shares
            balance = 0
            have_shares = 0


    return_rate = (return_rate-1) * 100.0
    
    accuracy = 0.0
    if(trade_times):
        accuracy = make_money/ trade_times * 100

    return return_rate, accuracy

## 4. 執行回測並搜集資料

In [56]:
random_choice_num = 10
data_directory = '../../parse_data/data/splited/'
ticker_name_file=  '../../parse_data/data/ticker_name.csv'
TIME_INTERVAL = 600

strategy_list = [bound_cross, middle_line, bound_cross_V, bound_cross_VM]
dictionary_rr = {}
dictionary_acc = {}

for data_file in file_list_with_directory(random_sample(ticker_list_file = ticker_name_file, times = random_choice_num), data_directory):
    
    slim_file_name = (data_file.split('/')[-1]).split('.')[0]

    print(data_file)
    ticker = get_dataframe(data_file, days=TIME_INTERVAL)
    ticker = kd_indicator(ticker)
    
    strategy_rr= []
    strategy_acc = []
    for i in range(len(strategy_list)):
        
        data = ticker.copy(deep=True)
        
        # initialize 'buy' & 'sell' column by filling with 0
        data['buy'] = np.zeros(ticker.shape[0])
        data['sell'] = np.zeros(ticker.shape[0])

        data = strategy_list[i](data)
        return_rate, acc = backtesting(data)
        
        if return_rate != np.nan:
            return_rate = (round(return_rate, 1))
            acc = (round(acc, 3))
        
        print('\treturn rate: {:>5.1f}% | acc: {:>5.1f}% | {}'.format(return_rate, acc, colored(strategy_list[i].__name__,  'red', attrs=['bold'])))
        strategy_rr.append((return_rate))
        strategy_acc.append((acc))
        print()
    

    #print(strategy_rr)
    #print(strategy_acc)
    
    dictionary_rr[slim_file_name] = strategy_rr
    dictionary_acc[slim_file_name] = strategy_acc



../../parse_data/data/splited/1733.csv
	buy  | 2019-06-20 | 31.6
	buy  | 2019-08-16 | 28.0
	buy  | 2020-01-13 | 27.5
	sell | 2020-04-13 | 22.3 * 3
	buy  | 2020-09-29 | 24.5
	buy  | 2020-10-23 | 23.95
	sell | 2020-11-16 | 24.45 * 2
	buy  | 2020-12-21 | 24.05
	buy  | 2021-01-12 | 22.8
	sell | 2021-03-02 | 23.25 * 2
	return rate: -23.1% | acc:  33.3% | bound_cross

	buy  | 2018-11-09 | 27.15
	buy  | 2018-11-30 | 28.35
	buy  | 2018-12-14 | 28.3
	sell | 2019-01-02 | 29.85 * 3
	buy  | 2019-01-28 | 30.9
	buy  | 2019-02-20 | 34.8
	sell | 2019-03-13 | 36.2 * 2
	buy  | 2019-05-13 | 32.25
	buy  | 2019-05-24 | 33.95
	sell | 2019-06-06 | 33.8 * 2
	buy  | 2019-07-09 | 31.7
	buy  | 2019-07-22 | 32.1
	sell | 2019-08-02 | 29.0 * 2
	buy  | 2019-08-23 | 28.1
	buy  | 2019-09-09 | 27.6
	buy  | 2019-09-25 | 27.45
	sell | 2019-10-14 | 27.1 * 3
	buy  | 2019-11-26 | 28.15
	buy  | 2019-12-09 | 28.7
	sell | 2019-12-20 | 28.0 * 2
	buy  | 2020-03-30 | 21.2
	buy  | 2020-04-14 | 22.8
	buy  | 2020-04-27 | 23.5
	sell 

## 5. 統計結果

In [57]:
import re
ticker_target = '0050'
df_target = readStock_file('../../parse_data/data/splited/{}.csv'.format(ticker_target))
df_target = df_target.tail(TIME_INTERVAL)
print('Compared target', colored(ticker_target, 'magenta'), ':', end=' ')
print(colored(str(round(((df_target['Close'][-1]/df_target['Close'][0])-1) * 100, 3)) +'%' , 'magenta',attrs=['bold']))
print()

extract_pattern = re.compile("(.+)%")

strategy_names = [i.__name__ for i in strategy_list]

dataframe_return_rate = pd.DataFrame.from_dict(dictionary_rr, orient='index',columns = strategy_names)
dataframe_return_rate_avg = dataframe_return_rate.mean(axis = 0, skipna = True).apply(lambda x : round(x, 3))
dataframe_return_rate_avg = (pd.DataFrame(dataframe_return_rate_avg, columns=['AVG']).transpose())
dataframe_return_rate = dataframe_return_rate.append(dataframe_return_rate_avg)

dataframe_acc = pd.DataFrame.from_dict(dictionary_acc ,orient='index',columns = strategy_names)
dataframe_acc_avg = dataframe_acc.mean(axis = 0, skipna = True).apply(lambda x : round(x, 3))
dataframe_acc_avg = (pd.DataFrame(dataframe_acc_avg, columns=['AVG']).transpose())
dataframe_acc = dataframe_acc.append(dataframe_acc_avg)

pattern = re.compile("(.+)%")
def adding_percent(df):

    for col in df.columns:
        df[col] = df[col].apply(lambda x : str(x) + '%')
    return df

def style_return(df):
    style = df.style.apply(lambda x : ['color:green' if ('nan' not in i and eval(pattern.match(i)[1]) > 1) else '' for i in x])
    style = style.apply(lambda x : ['color:red' if ('nan' not in i and -1.0 > eval(pattern.match(i)[1])) else '' for i in x])
    
    return style
def style_win(df):
    style = df.style.apply(lambda x : ['color:red' if ('nan' not in i  and eval(pattern.match(i)[1]) < 50) else '' for i in x])
    style = style.apply(lambda x : ['color:green' if ('nan' not in i and eval(pattern.match(i)[1]) > 50) else '' for i in x])

    return style




def style_avg(style):
    def custom_avg_row(row):

        # Underline + bold the AVG
        colors = ['background-color: \'\'']*len(row.index)
        colors[-1] = 'font-weight: bold;text-decoration: underline;'
        return colors
    
    style = style.apply(custom_avg_row, axis=0)
    
    # change header color
    headers = {
    'selector': 'th:not(.index_name)',
    'props': [('background-color', '#fff9c7')]
    }
    style = style.set_table_styles([headers])
    
    return style
 

from IPython.display import display

dataframe_return_rate = adding_percent(dataframe_return_rate)
adding_percent(dataframe_acc)
print('return rate:')
display(style_avg(style_return(dataframe_return_rate)))

print()
print('accuracy:')
display(style_avg(style_win(dataframe_acc)))



Compared target 0050 : 60.995%

return rate:


,bound_cross,middle_line,bound_cross_V,bound_cross_VM
1733,-23.1%,5.5%,-13.0%,-18.2%
1538,174.8%,-8.0%,118.3%,-6.1%
1515,4.0%,-14.3%,41.2%,-12.2%
2067,25.5%,-21.2%,4.2%,-31.3%
2397,46.7%,-12.2%,30.9%,-21.9%
4188,-12.8%,4.2%,3.1%,5.5%
52634,24.0%,1.5%,38.5%,-0.3%
3686,227.5%,13.1%,59.7%,23.3%
54755,nan%,nan%,nan%,nan%
2597,30.4%,17.3%,112.8%,42.1%



accuracy:


,bound_cross,middle_line,bound_cross_V,bound_cross_VM
1733,33.333%,50.0%,71.429%,33.333%
1538,75.0%,40.0%,80.0%,41.667%
1515,100.0%,26.667%,62.5%,46.154%
2067,66.667%,22.222%,66.667%,27.273%
2397,100.0%,35.294%,77.778%,33.333%
4188,25.0%,31.25%,42.857%,30.0%
52634,100.0%,100.0%,100.0%,0.0%
3686,60.0%,22.222%,62.5%,37.5%
54755,nan%,nan%,nan%,nan%
2597,75.0%,53.333%,76.923%,40.0%
